# CPE Data

In [1]:
import glob
import logging
import json
import os
import pandas as pd
import warnings

logging.getLogger('matplotlib.font_manager').disabled = True
warnings.filterwarnings("ignore")

In [2]:
row_accumulator = []

for filename in glob.glob('nvdcve-1.1-*.json'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for cves in nvd_data['CVE_Items']:
            if cves['configurations']['nodes'] != []:
                if "children" in cves['configurations']['nodes'][0]:
                    for j in range(len(cves['configurations']['nodes'][0]['children'])):
                        for cpe in cves['configurations']['nodes'][0]['children'][j]['cpe_match']:
                            if cpe['vulnerable'] == True:
                                        cve = cves['cve']['CVE_data_meta']['ID']
                                        cpe_string = cpe['cpe23Uri']
                                        try:
                                            end = cpe['versionEndExcluding']
                                        except: 
                                            end = 'None'
                                        try:
                                            end2 = cpe['versionEndIncluding']
                                        except: 
                                            end2 = 'None' 
                                        try:
                                            start = cpe['versionStartExcluding']
                                        except: 
                                            start = 'None'
                                        try:
                                            start2 = cpe['versionStartIncluding']
                                        except: 
                                            start2 = 'None'     
                                        new_row = { 
                                        'CVE': cve,
                                        'CPE' : cpe_string,
                                        'StartI' : start,
                                        'StartE' : start2,
                                        'EndI' : end,
                                        'EndE' : end2 
                                        }
                                        row_accumulator.append(new_row)
        


nvd = pd.DataFrame(row_accumulator)                            

## CVEs With Most CPE

In [3]:
nvd['CVE'].value_counts().head(20)

CVE-2021-1620     690
CVE-2022-22160    364
CVE-2011-3664     325
CVE-2012-1350     322
CVE-2007-6694     316
CVE-2008-7256     303
CVE-2011-1011     270
CVE-2021-31366    267
CVE-2012-5120     248
CVE-2016-5995     243
CVE-2010-0741     241
CVE-2022-22226    240
CVE-2021-31382    239
CVE-2021-0272     235
CVE-2021-0228     231
CVE-2020-1680     228
CVE-2021-0273     226
CVE-2009-2711     226
CVE-2011-3666     225
CVE-2011-2366     225
Name: CVE, dtype: int64

## Most Common CPEs

In [4]:
nvd['CPE'].value_counts().head(20)

cpe:2.3:a:adobe:acrobat_dc:*:*:*:*:classic:*:*:*                         2009
cpe:2.3:a:adobe:acrobat_reader_dc:*:*:*:*:classic:*:*:*                  2008
cpe:2.3:a:adobe:acrobat_dc:*:*:*:*:continuous:*:*:*                      1438
cpe:2.3:a:adobe:acrobat_reader_dc:*:*:*:*:continuous:*:*:*               1432
cpe:2.3:a:adobe:flash_player:*:*:*:*:*:*:*:*                              714
cpe:2.3:a:adobe:acrobat:*:*:*:*:*:*:*:*                                   651
cpe:2.3:a:google:chrome:*:*:*:*:*:*:*:*                                   389
cpe:2.3:a:foxitsoftware:phantompdf:*:*:*:*:*:*:*:*                        374
cpe:2.3:a:adobe:reader:*:*:*:*:*:*:*:*                                    356
cpe:2.3:o:google:android:11.0:*:*:*:*:*:*:*                               317
cpe:2.3:o:qualcomm:apq8009_firmware:-:*:*:*:*:*:*:*                       301
cpe:2.3:a:adobe:acrobat_reader:*:*:*:*:*:*:*:*                            298
cpe:2.3:o:qualcomm:mdm9206_firmware:-:*:*:*:*:*:*:*             